### Extract Features
Here we will be using Mel-Frequency Cepstral Coefficients(MFCC) from the audio 
samples.
The MFCC summarises the frequency distribution across the window size, so it is possible to analyse both the frequency and time characteristics of the sound. These audio representations will allow us to identify features for classification.

In [40]:
#### Extracting MFCC's For every audio file
import pandas as pd
import os
import librosa

audio_dataset_path='college_dataset/audio'

metadata=pd.read_csv('college_dataset/metadata/college_dataset.csv')

#metadata.tail()


,slice_file_name,classID,class,fold
1429,hostel56.wav,16,hostel,17
1430,hostel57.wav,16,hostel,17
1431,hostel58.wav,16,hostel,17
1432,hostel59.wav,16,hostel,17
1433,hostel60.wav,16,hostel,17


In [41]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    return mfccs_scaled_features
    

In [42]:
import numpy as np
from tqdm import tqdm
### Now we iterate through every audio file and extract features 
### using Mel-Frequency Cepstral Coefficients
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    final_class_labels=row["class"]
   #print(file_name)
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])


1434it [00:30, 47.30it/s]


In [43]:
### converting extracted_features to Pandas dataframe
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-335.68454, 122.79147, -26.744972, 31.406775,...",Introduce
1,"[-330.10046, 121.88401, -21.08908, 51.432224, ...",Introduce
2,"[-330.10046, 121.88401, -21.08908, 51.432224, ...",Introduce
3,"[-330.10046, 121.88401, -21.08908, 51.432224, ...",Introduce
4,"[-330.10046, 121.88401, -21.08908, 51.432224, ...",Introduce


In [44]:
### Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [45]:
### Label Encoding
###y=np.array(pd.get_dummies(y))
### Label Encoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [46]:
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

### Model Creation

In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [48]:
### No of classes
num_labels=y.shape[1]
print(num_labels)

17


In [49]:
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
# model.add(Conv2D(100, kernel_size=3, activation='relu', input_shape=(40,)))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [50]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               4100      
                                                                 
 activation_4 (Activation)   (None, 100)               0         
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_5 (Dense)             (None, 200)               20200     
                                                                 
 activation_5 (Activation)   (None, 200)               0         
                                                                 
 dropout_4 (Dropout)         (None, 200)               0         
                                                                 
 dense_6 (Dense)             (None, 100)              

In [51]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [52]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 500
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification_collegeDataset.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/500
30/36 [========================>.....] - ETA: 0s - loss: 55.4500 - accuracy: 0.0562 
Epoch 1: val_loss improved from inf to 3.27006, saving model to saved_models\audio_classification_collegeDataset.hdf5
36/36 [==============================] - 1s 14ms/step - loss: 50.6250 - accuracy: 0.0558 - val_loss: 3.2701 - val_accuracy: 0.0244
Epoch 2/500
29/36 [=======================>......] - ETA: 0s - loss: 17.7928 - accuracy: 0.0647
Epoch 2: val_loss improved from 3.27006 to 2.83812, saving model to saved_models\audio_classification_collegeDataset.hdf5
36/36 [==============================] - 0s 11ms/step - loss: 16.7583 - accuracy: 0.0610 - val_loss: 2.8381 - val_accuracy: 0.1010
Epoch 3/500
26/36 [====================>.........] - ETA: 0s - loss: 8.9356 - accuracy: 0.0721
Epoch 3: val_loss improved from 2.83812 to 2.83342, saving model to saved_models\audio_classification_collegeDataset.hdf5
36/36 [==============================] - 0s 7ms/step - loss: 8.4178 - accuracy: 0.0706 -

36/36 [==============================] - 0s 8ms/step - loss: 2.8213 - accuracy: 0.0724 - val_loss: 2.8048 - val_accuracy: 0.0488
Epoch 25/500
32/36 [=========================>....] - ETA: 0s - loss: 2.8156 - accuracy: 0.0781
Epoch 25: val_loss improved from 2.80482 to 2.80449, saving model to saved_models\audio_classification_collegeDataset.hdf5
36/36 [==============================] - 0s 7ms/step - loss: 2.8157 - accuracy: 0.0759 - val_loss: 2.8045 - val_accuracy: 0.0488
Epoch 26/500
31/36 [========================>.....] - ETA: 0s - loss: 2.8121 - accuracy: 0.0766
Epoch 26: val_loss improved from 2.80449 to 2.80401, saving model to saved_models\audio_classification_collegeDataset.hdf5
36/36 [==============================] - 0s 6ms/step - loss: 2.8174 - accuracy: 0.0785 - val_loss: 2.8040 - val_accuracy: 0.0488
Epoch 27/500
31/36 [========================>.....] - ETA: 0s - loss: 2.8112 - accuracy: 0.0706
Epoch 27: val_loss improved from 2.80401 to 2.80350, saving model to saved_mode

25/36 [===================>..........] - ETA: 0s - loss: 2.8180 - accuracy: 0.0800
Epoch 50: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 5ms/step - loss: 2.8029 - accuracy: 0.0785 - val_loss: 2.8017 - val_accuracy: 0.0488
Epoch 51/500
26/36 [====================>.........] - ETA: 0s - loss: 2.8057 - accuracy: 0.0733
Epoch 51: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 6ms/step - loss: 2.8006 - accuracy: 0.0759 - val_loss: 2.8015 - val_accuracy: 0.0488
Epoch 52/500
30/36 [========================>.....] - ETA: 0s - loss: 2.7863 - accuracy: 0.0792
Epoch 52: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 6ms/step - loss: 2.7923 - accuracy: 0.0759 - val_loss: 2.8019 - val_accuracy: 0.0488
Epoch 53/500
32/36 [=========================>....] - ETA: 0s - loss: 2.7904 - accuracy: 0.0781
Epoch 53: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 5ms/st

30/36 [========================>.....] - ETA: 0s - loss: 2.7871 - accuracy: 0.0792  
Epoch 80: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 5ms/step - loss: 2.7899 - accuracy: 0.0759 - val_loss: 2.8023 - val_accuracy: 0.0488
Epoch 81/500
33/36 [==========================>...] - ETA: 0s - loss: 2.7892 - accuracy: 0.0786
Epoch 81: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 5ms/step - loss: 2.7944 - accuracy: 0.0759 - val_loss: 2.8021 - val_accuracy: 0.0488
Epoch 82/500
25/36 [===================>..........] - ETA: 0s - loss: 2.7991 - accuracy: 0.0750
Epoch 82: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 5ms/step - loss: 2.7938 - accuracy: 0.0767 - val_loss: 2.8020 - val_accuracy: 0.0488
Epoch 83/500
33/36 [==========================>...] - ETA: 0s - loss: 2.7899 - accuracy: 0.0767
Epoch 83: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 8ms/

Epoch 110/500
29/36 [=======================>......] - ETA: 0s - loss: 2.7852 - accuracy: 0.0711  
Epoch 110: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 5ms/step - loss: 2.7907 - accuracy: 0.0767 - val_loss: 2.8020 - val_accuracy: 0.0488
Epoch 111/500
32/36 [=========================>....] - ETA: 0s - loss: 2.7888 - accuracy: 0.0781
Epoch 111: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 5ms/step - loss: 2.7891 - accuracy: 0.0767 - val_loss: 2.8023 - val_accuracy: 0.0488
Epoch 112/500
31/36 [========================>.....] - ETA: 0s - loss: 2.7887 - accuracy: 0.0766
Epoch 112: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 5ms/step - loss: 2.7896 - accuracy: 0.0767 - val_loss: 2.8023 - val_accuracy: 0.0488
Epoch 113/500
26/36 [====================>.........] - ETA: 0s - loss: 2.7887 - accuracy: 0.0805
Epoch 113: val_loss did not improve from 2.80148
36/36 [====================

...

Epoch 380/500
30/36 [========================>.....] - ETA: 0s - loss: 2.7905 - accuracy: 0.0792
Epoch 380: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 9ms/step - loss: 2.7888 - accuracy: 0.0767 - val_loss: 2.8024 - val_accuracy: 0.0488
Epoch 381/500
33/36 [==========================>...] - ETA: 0s - loss: 2.7881 - accuracy: 0.0824
Epoch 381: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 7ms/step - loss: 2.7888 - accuracy: 0.0767 - val_loss: 2.8026 - val_accuracy: 0.0488
Epoch 382/500
28/36 [======================>.......] - ETA: 0s - loss: 2.7857 - accuracy: 0.0770
Epoch 382: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 8ms/step - loss: 2.7893 - accuracy: 0.0767 - val_loss: 2.8025 - val_accuracy: 0.0488
Epoch 383/500
28/36 [======================>.......] - ETA: 0s - loss: 2.7872 - accuracy: 0.0826
Epoch 383: val_loss did not improve from 2.80148
36/36 [======================

Epoch 410/500
31/36 [========================>.....] - ETA: 0s - loss: 2.7850 - accuracy: 0.0766
Epoch 410: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 5ms/step - loss: 2.7888 - accuracy: 0.0767 - val_loss: 2.8022 - val_accuracy: 0.0488
Epoch 411/500
27/36 [=====================>........] - ETA: 0s - loss: 2.7921 - accuracy: 0.0729
Epoch 411: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 6ms/step - loss: 2.7888 - accuracy: 0.0767 - val_loss: 2.8024 - val_accuracy: 0.0488
Epoch 412/500
32/36 [=========================>....] - ETA: 0s - loss: 2.7866 - accuracy: 0.0762
Epoch 412: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 5ms/step - loss: 2.7888 - accuracy: 0.0767 - val_loss: 2.8024 - val_accuracy: 0.0488
Epoch 413/500
25/36 [===================>..........] - ETA: 0s - loss: 2.7869 - accuracy: 0.0675
Epoch 413: val_loss did not improve from 2.80148
36/36 [======================

Epoch 440/500
28/36 [======================>.......] - ETA: 0s - loss: 2.7851 - accuracy: 0.0748
Epoch 440: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 5ms/step - loss: 2.7888 - accuracy: 0.0767 - val_loss: 2.8024 - val_accuracy: 0.0488
Epoch 441/500
29/36 [=======================>......] - ETA: 0s - loss: 2.7906 - accuracy: 0.0722
Epoch 441: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 5ms/step - loss: 2.7888 - accuracy: 0.0767 - val_loss: 2.8024 - val_accuracy: 0.0488
Epoch 442/500
32/36 [=========================>....] - ETA: 0s - loss: 2.7928 - accuracy: 0.0781
Epoch 442: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 5ms/step - loss: 2.7884 - accuracy: 0.0767 - val_loss: 2.8025 - val_accuracy: 0.0488
Epoch 443/500
31/36 [========================>.....] - ETA: 0s - loss: 2.7878 - accuracy: 0.0766
Epoch 443: val_loss did not improve from 2.80148
36/36 [======================

Epoch 470/500
27/36 [=====================>........] - ETA: 0s - loss: 2.7916 - accuracy: 0.0729
Epoch 470: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 5ms/step - loss: 2.7888 - accuracy: 0.0767 - val_loss: 2.8024 - val_accuracy: 0.0488
Epoch 471/500
29/36 [=======================>......] - ETA: 0s - loss: 2.7854 - accuracy: 0.0787
Epoch 471: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 5ms/step - loss: 2.7888 - accuracy: 0.0767 - val_loss: 2.8023 - val_accuracy: 0.0488
Epoch 472/500
31/36 [========================>.....] - ETA: 0s - loss: 2.7925 - accuracy: 0.0766
Epoch 472: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 5ms/step - loss: 2.7888 - accuracy: 0.0767 - val_loss: 2.8024 - val_accuracy: 0.0488
Epoch 473/500
32/36 [=========================>....] - ETA: 0s - loss: 2.7898 - accuracy: 0.0762
Epoch 473: val_loss did not improve from 2.80148
36/36 [======================

Epoch 500/500
30/36 [========================>.....] - ETA: 0s - loss: 2.7866 - accuracy: 0.0812
Epoch 500: val_loss did not improve from 2.80148
36/36 [==============================] - 0s 5ms/step - loss: 2.7888 - accuracy: 0.0767 - val_loss: 2.8023 - val_accuracy: 0.0488
Training completed in time:  0:01:48.479589


In [53]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1]*100)

4.878048598766327


In [54]:
import pyaudio
import wave

chunk = 1024  # Record in chunks of 1024 samples
sample_format = pyaudio.paInt16  # 16 bits per sample
channels = 2
fs = 44100  # Record at 44100 samples per second
seconds = 6
filename = "record/output.wav"

p = pyaudio.PyAudio()  # Create an interface to PortAudio

print('Recording.....')

stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input=True)

frames = []  # Initialize array to store frames

# Store data in chunks for 3 seconds
for i in range(0, int(fs / chunk * seconds)):
    data = stream.read(chunk)
    frames.append(data)

# Stop and close the stream 
stream.stop_stream()
stream.close()
# Terminate the PortAudio interface
p.terminate()

print('Finished recording')

# Save the recorded data as a WAV file
wf = wave.open(filename, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(fs)
wf.writeframes(b''.join(frames))
wf.close()

Recording.....
Finished recording


### Testing Some Test Audio Data

Steps
- Preprocess the new audio data
- predict the classes
- Invere transform your Predicted Label

In [55]:
filename="record/output.wav"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
print(mfccs_scaled_features)
print(mfccs_scaled_features.shape)
predicted_label=np.argmax(model.predict(mfccs_scaled_features),axis=1)
#predicted_label=model.predict_classes(mfccs_scaled_features)
print(predicted_label)
prediction_class = labelencoder.inverse_transform(predicted_label) 
prediction_class[0]

[-5.2398022e+02  1.5536185e+02 -3.6779476e+01 -1.0843380e+01
 -2.1722378e+01 -2.4120514e+01 -2.3861486e+01 -1.7462698e+01
 -5.5346198e+00 -1.4216748e+01 -9.3451204e+00 -2.4441979e+00
 -5.0089378e+00 -8.2656088e+00 -6.1156945e+00 -4.1016989e+00
 -3.3338749e+00 -6.9545369e+00 -1.3932248e+01 -8.3481750e+00
 -2.7037563e+00 -4.0025234e+00 -1.3923370e+00 -2.5312581e+00
 -9.1694803e+00 -1.0224369e+01 -4.0654016e+00 -3.7894495e+00
 -1.3115091e+00  3.3021412e+00 -3.7751281e+00 -3.5275407e+00
 -4.2646332e+00 -5.3335447e+00  1.5227566e+00  1.7903449e-02
 -3.6879715e-01 -2.0127854e+00 -2.6375427e+00  5.8087051e-01]
[[-5.2398022e+02  1.5536185e+02 -3.6779476e+01 -1.0843380e+01
  -2.1722378e+01 -2.4120514e+01 -2.3861486e+01 -1.7462698e+01
  -5.5346198e+00 -1.4216748e+01 -9.3451204e+00 -2.4441979e+00
  -5.0089378e+00 -8.2656088e+00 -6.1156945e+00 -4.1016989e+00
  -3.3338749e+00 -6.9545369e+00 -1.3932248e+01 -8.3481750e+00
  -2.7037563e+00 -4.0025234e+00 -1.3923370e+00 -2.5312581e+00
  -9.1694803e+00 

'feestructure'